Developing machine learning models is usually an iterative process. You start with an initial design then reconfigure until you get a model that can be trained efficiently in terms of time and compute resources. As you may already know, these settings that you adjust are called _hyperparameters_. These are the variables that govern the training process and the topology of an ML model. These remain constant over the training process and directly impact the performance of your ML program. 

The process of finding the optimal set of hyperparameters is called *hyperparameter tuning* or *hypertuning*, and it is an essential part of a machine learning pipeline. Without it, you might end up with a model that has unnecessary parameters and take too long to train.

Hyperparameters are of two types:
1. *Model hyperparameters* which influence model selection such as the number and width of hidden layers

2. *Algorithm hyperparameters* which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier.

For more complex models, the number of hyperparameters can increase dramatically and tuning them manually can be quite challenging.


hyperparameter tuning with [Keras Tuner](https://keras-team.github.io/keras-tuner/), a package from the Keras team that automates this process.

## Keras Tuner

To perform hypertuning with Keras Tuner, you will need to:

* Define the model
* Select which hyperparameters to tune
* Define its search space
* Define the search strategy

https://colab.research.google.com/github/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course3/week1-ungraded-lab/C3_W1_Lab_1_Keras_Tuner.ipynb#scrollTo=BjVYPOw6MH5d

## TFX Pipeline

With the setup complete, you can now proceed to creating the pipeline. 

In [9]:
import tensorflow as tf
import os
import tfx
from tfx.components import ImportExampleGen , CsvExampleGen
from tfx.orchestration import metadata
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip



The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [10]:
base_dir = os.getcwd()
Tf_record_data_dir = os.path.join(os.pardir, "tfrecord_data")
tf_record_location = os.path.join(base_dir, Tf_record_data_dir)
tf_record_location

'C:\\Users\\ASUS\\building-machine-learning-pipelines\\Untitled Folder\\..\\tfrecord_data'

In [11]:
example_gen = ImportExampleGen(input_base=tf_record_location)#ImportExampleGen because we are impoting Tfrecord file format

In [12]:
context = InteractiveContext(pipeline_root='../pipeline_root_learning/tfx_for_tfrecordData')   # ".." means the parent directory
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 292
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [13]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 293
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
from tfx.components import SchemaGen 
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 294
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Transform

In [15]:
_housing_transform_module_file = "housing_transform.py"

In [16]:
%%writefile {_housing_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'ocean_proximity'
]

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['longitude','latitude','total_rooms','total_bedrooms','population','households','median_income','median_house_value']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['housing_median_age']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'housing_median_age':6}

# Feature that the model will predict
LABEL_KEY = 'ocean_proximity'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting {_housing_constants_module_file}


In [17]:
_housing_transform_module_file = 'housing_transform.py'

In [18]:
%%writefile {_housing_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import housing_constant

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = housing_constant.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = housing_constant.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = housing_constant.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = housing_constant.FEATURE_BUCKET_COUNT
_LABEL_KEY = housing_constant.LABEL_KEY
_transformed_name = housing_constant.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1( # appending transformed features into empy dictionary
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Overwriting housing_transform.py


In [19]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'], # tfExample
    schema=schema_gen.outputs['schema'],# Schema
    module_file=os.path.abspath(_housing_transform_module_file)) # preprocessing_fn()



#### TFtransform

In [20]:
# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 295
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Tuner


As the name suggests, the [Tuner](https://www.tensorflow.org/tfx/guide/tuner) component tunes the hyperparameters of your model. To use this, you will need to provide a *tuner module file* which contains a `tuner_fn()` function. In this function, you will mostly do the same steps as you did in the previous ungraded lab(
https://colab.research.google.com/github/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course3/week1-ungraded-lab/C3_W1_Lab_1_Keras_Tuner.ipynb#scrollTo=BjVYPOw6MH5d) but with some key differences in handling the dataset. 

The Transform component earlier saved the transformed examples as TFRecords compressed in `.gz` format and you will need to load that into memory. Once loaded, you will need to create batches of features and labels so you can finally use it for hypertuning. This process is modularized in the `_input_fn()` below. 

Going back, the `tuner_fn()` function will return a `TunerFnResult` [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) containing your `tuner` object and a set of arguments to pass to `tuner.search()` method. You will see these in action in the following cells. When reviewing the module file, we recommend viewing the `tuner_fn()` first before looking at the other auxiliary functions.

In [21]:
# Declare name of module file
_tuner_module_file = 'tuner.py'

In [22]:
%%writefile {_tuner_module_file}
# Import necessary libraries
from kerastuner.engine import base_tuner
import kerastuner as kt
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs
import tensorflow as tf
import tensorflow_transform as tft

# Constants from housing_constants_module_file
import housing_constant
_transformed_name = housing_constant.transformed_name


TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

LABEL_KEY = "ocean_proximity_xf"


def _gzip_reader_fn(filenames):
  '''Load compressed dataset
  
  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''

  # Loading the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output - transform output graph
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''

  # Get feature specification based on transform output
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  # Create batches of features and labels
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset

def build_model(hp):
    input_features = []
    for x in housing_constant.NUMERIC_FEATURE_KEYS:
        input_features.append(tf.keras.Input(shape=(1,), name=_transformed_name(x)))

    for x in housing_constant.BUCKET_FEATURE_KEYS:
        input_features.append(tf.keras.Input(shape=(1,), name=_transformed_name(x)))  # Bucket features have shape (1,) after transformation

    # Concatenate all input layers
    concat = tf.keras.layers.concatenate(input_features)

    # Hidden layers
    first_layer = tf.keras.layers.Dense(units=hp.Int('units_1', min_value=50, max_value=100, step=5), activation='relu')(concat)
    dropout_layer = tf.keras.layers.Dropout(0.2)(first_layer)
    second_layer = tf.keras.layers.Dense(units=hp.Int('units_2', min_value=125, max_value=250, step=25), activation='relu')(dropout_layer)
    third_layer = tf.keras.layers.Dense(units=hp.Int('units_3', min_value=200, max_value=550, step=25), activation='relu')(second_layer)


    # Output layer
    output_layer = tf.keras.layers.Dense(units=5, activation='softmax')(third_layer)

    # Create the model
    model = tf.keras.Model(inputs=input_features, outputs=output_layer)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy', 
                  metrics=['categorical_accuracy'])
    
    return model


def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args as name/value pairs.

    Returns:
        A BaseTuner object that will be used for tuning.
    """ 

    # Initialize the tuner
    tuner = kt.Hyperband(build_model,
                         objective='val_loss',
                         max_epochs=15,
                         factor=3,
                         directory=fn_args.working_dir,
                         project_name='housing_tuning')

    # Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    # Use _input_fn() to extract input features and labels from the train and val set
    train_set = _input_fn(fn_args.train_files, tf_transform_output)
    val_set = _input_fn(fn_args.eval_files, tf_transform_output)


    return TunerFnResult(
      tuner=tuner,
      fit_kwargs={ 
          'x': train_set,
          'validation_data': val_set,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      }
  )



Overwriting tuner.py


In [23]:
# Setup the Tuner component  /// https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Tuner
tuner = tfx.components.Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=512),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )

In [24]:
# Run the component. This will take around 10 minutes to run.
# When done, it will summarize the results and show the 10 best trials.
context.run(tuner, enable_cache=False)

Trial 30 Complete [00h 00m 32s]
val_loss: 0.9787499904632568

Best val_loss So Far: 0.973437488079071
Total elapsed time: 00h 05m 40s
Results summary
Results in ../pipeline_root_learning/tfx_for_tfrecordData\.temp\296\housing_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0026 summary
Hyperparameters:
units_1: 90
units_2: 150
units_3: 275
learning_rate: 0.01
tuner/epochs: 15
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.973437488079071

Trial 0029 summary
Hyperparameters:
units_1: 80
units_2: 200
units_3: 375
learning_rate: 0.0001
tuner/epochs: 15
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9787499904632568

Trial 0027 summary
Hyperparameters:
units_1: 65
units_2: 200
units_3: 250
learning_rate: 0.001
tuner/epochs: 15
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9806249737739563

Trial 0016 summary
Hyperparameters:
units_1: 70
units_2: 175
units_3: 525
learning_rate: 0.01
tuner/epochs: 15
tuner/i

TypeError: '<' not supported between instances of 'int' and 'NoneType'

In [175]:
tuner

Tuner(spec: <tfx.types.standard_component_specs.TunerSpec object at 0x000001FC5FB47580>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x000001FC5FB47A90>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Tuner, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'best_hyperparameters': OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'tuner_results': OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None})